In [ ]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values
COS_ENDPOINT = "https://s3.us-south.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
COS_API_KEY_ID = "###################################" # eg "W00YiRnLW4a3fTjMB-odB-2ySfTrFBIQQWanc--P3byk"
COS_AUTH_ENDPOINT = "https://iam.cloud.ibm.com/identity/token"
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/##############::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"

# COS Credentials
cos = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

def get_buckets():
    print("Retrieving list of buckets")
    try:
        buckets = cos.buckets.all()
        for bucket in buckets:
            print("Bucket Name: {0}".format(bucket.name))
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to retrieve list buckets: {0}".format(e))

def multi_part_upload(bucket_name, item_name, file_path):
    try:
        print("Starting file transfer for {0} to bucket: {1}\n".format(item_name, bucket_name))
        # set 5 MB chunks
        part_size = 1024 * 1024 * 5

        # set threadhold to 15 MB
        file_threshold = 1024 * 1024 * 15

        # set the transfer threshold and chunk size
        transfer_config = ibm_boto3.s3.transfer.TransferConfig(
            multipart_threshold=file_threshold,
            multipart_chunksize=part_size
        )

        # the upload_fileobj method will automatically execute a multi-part upload
        # in 5 MB chunks for all files over 15 MB
        with open(file_path, "rb") as file_data:
            cos.Object(bucket_name, item_name).upload_fileobj(
                Fileobj=file_data,
                Config=transfer_config
            )

        print("Transfer for {0} Complete!\n".format(item_name))
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to complete multi-part upload: {0}".format(e))
        
    
get_buckets()

In [ ]:
!pip install tweepy

In [ ]:
states_array = ["Aguascalientes","Baja California","Baja California Sur","Campeche","Coahuila de Zaragoza","Colima","Chiapas","Chihuahua","Distrito Federal","Durango","Guanajuato","Guerrero","Hidalgo","Jalisco","México","Michoacán de Ocampo","Morelos","Nayarit","Nuevo León","Oaxaca","Puebla","Querétaro Arteaga","Quintana Roo","San Luis Potosí","Sinaloa","Sonora","Tabasco","Tamaulipas","Tlaxcala","Veracruz de Ignacio de la Llave","Yucatán","Zacatecas"]
states_data = {
"Aguascalientes":[],
"Baja California":[],
"Baja California Sur":[],
"Campeche":[],
"Coahuila de Zaragoza":[],
"Colima":[],
"Chiapas":[],
"Chihuahua":[],
"Distrito Federal":[],
"Durango":[],
"Guanajuato":[],
"Guerrero":[],
"Hidalgo":[],
"Jalisco":[],
"México":[],
"Michoacán de Ocampo":[],
"Morelos":[],
"Nayarit":[],
"Nuevo León":[],
"Oaxaca":[],
"Puebla":[],
"Querétaro Arteaga":[],
"Quintana Roo":[],
"San Luis Potosí":[],
"Sinaloa":[],
"Sonora":[],
"Tabasco":[],
"Tamaulipas":[],
"Tlaxcala":[],
"Veracruz de Ignacio de la Llave":[],
"Yucatán":[],
"Zacatecas":[]
}

In [ ]:
from __future__ import absolute_import, print_function
from tweepy import OAuthHandler, Stream, StreamListener
import json
import requests
import base64

# Twitter

consumer_key="################################"
consumer_secret="#############################"
access_token="################################"
access_token_secret="#########################"

# Tone Analyzer 

APIkey = "####################################"
ToneURL = "###################################"

# Translate API

YandexAPI="###################################"

# Bucket Name

bucket = "YOURBUCKETNAME"

# Code:

APIkey ="apikey:"+APIkey
message_bytes = APIkey.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

countpos=0
countneg=0

state=0
counter=0

def uploadvalues():
    datajson=states_data
    datasum = {"0":[],"1":[],"2":[],"3":[],"4":[],"5":[],"6":[],"7":[],"8":[],"9":[],"10":[],"11":[],"12":[],"13":[],"14":[],"15":[],"16":[],"17":[],"18":[],"19":[],"20":[],"21":[],"22":[],"23":[],"24":[],"25":[],"26":[],"27":[],"28":[],"29":[],"30":[],"31":[]}
    counter = 0
    for x in datajson:
        sumdata=0
        sumdatac=0
        for y in datajson[x]:
            sumdata=sumdata+y
            sumdatac=sumdatac+1
        datasum[str(counter)].append(sumdata)
        datasum[str(counter)].append(sumdatac)
        counter+=1
    datasum = str(datasum).replace("'", '"')
    text_file = open("sentimient.json", "w")
    n = text_file.write(datasum)
    text_file.close()
    multi_part_upload(bucket,"sentimient.json","sentimient.json")   
    
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

class StdOutListener(StreamListener):
    def on_data(self, data):
        global counter
        if(counter>99):  
            uploadvalues()
            counter=0
        counter=counter+1
        data = json.loads(data)
        if(data["place"]["country_code"]=="MX"):
            tempstring=data["place"]["full_name"]
            comma_place = tempstring.find(', ')
            tempstring = tempstring[comma_place+2:]
            for x in states_array:
                if(tempstring==x):
                    data = data["text"]
                    payload = data
                    url = "https://translate.yandex.net/api/v1.5/tr.json/translate?key="+YandexAPI+"&text=+"+payload+"&lang=es-en"
                    payload = {}
                    headers = {}
                    response = requests.request("GET", url, headers=headers, data = payload)
                    response = json.loads(response.text.encode('utf8').decode())
                    if(response["code"]==200):
                        response = deEmojify(response["text"][0])
                        tweet=response
                        url = ToneURL +"/v3/tone?version=2017-09-21&sentences=false"
                        
                        payload = '{\r\n  \"text\": \"'+response+'\"\r\n}'

                        headers = {
                          'Content-Type': 'application/json',
                          'Content-Language': 'en-us',
                          'Accept-Language': 'en',
                          'Authorization': 'Basic '+base64_message,
                          'Content-Type': 'text/plain'
                        }
                        
                        response = requests.request("POST", url, headers=headers, data = payload)
                        response = json.loads(response.text.encode('utf8').decode())
                        response = response["document_tone"]["tones"]
                        
                        if(len(response)==0):
                            break
                        else:
                            temp=0
                            temps=""
                            flag =0
                            for y in response:
                                if(y["score"]>temp):
                                    temp=y["score"]
                                    temps=y
                            if(temps['tone_id']=="confident"):
                                flag =1
                            elif(temps['tone_id']=="sadness"):
                                flag =0
                            elif(temps['tone_id']=="tentative"):
                                flag =1
                            elif(temps['tone_id']=="fear"):
                                flag =0
                            elif(temps['tone_id']=="joy"):
                                flag =1
                            elif(temps['tone_id']=="analytical"):
                                flag =1
                            elif(temps['tone_id']=="anger"):
                                flag =0
                            states_data[x].append(flag)
                            print(tweet+ " " + str(flag))
                        break
            return True
    def on_error(self, status):
        print(status)

if __name__ == '__main__':
    while 1:
        try:
            l = StdOutListener()
            auth = OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_token_secret)
            stream = Stream(auth, l)
            stream.filter(locations=[-119.926760,14.242988,-86.150394,32.895289])
        except:
            print("Tweet not captured")